# Multi-Json

In [1]:
import json
import csv
import os
import argparse

In [2]:
results_dir = "./results"

# Reads all of the json files in the dir
def list_json_files(directory):
    json_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.json'):
                json_files.append(os.path.join(root, file))
    return json_files

list_dir = list_json_files(results_dir)


In [32]:
def above_two(data_records):
    index=[]
    for i in range(len(data_records)):
        modules = data_records[i]["modules"].keys()
        if len(modules) > 2:
            # print(i, len(modules))
            index.append(i)
        else:
            continue
    return index

def process_files(file_paths):
    
    csv_filename = 'comb_output.csv'
    
    # Check if the output file already exists
    file_exists = os.path.isfile(csv_filename)
    
    with open(csv_filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        
        if not file_exists:
            writer.writerow(["Bin_ID", "Contig_ID", "Gene_Coordinates", "Ctg_Number", "Ctg_Tag_Id", "Mibig_biosynthetic_gene_cluster", "Mibig_Product", "Blast_name", "Blast_genecluster","Blast_annotation","Blast_perc_coverage", "Blast_perc_ident", "Blast_blastscore", "Blast_evalue", "Blast_locus_tag" ])
        
        for file_path in file_paths:

            with open(file_path, 'r') as files:
                data = json.load(files)
                index_ = above_two(data["records"])
                
                for i in index_:
                    bin_id = data["input_file"]
                    contig_id = data["records"][i]["modules"]["antismash.modules.clusterblast"]["record_id"]
                    blast_info = data["records"][i]["modules"]["antismash.modules.clusterblast"]["general"]["results"][0]["ranking"][1][1]["pairings"]
                    mibig_entries = data["records"][i]["modules"]["antismash.modules.clusterblast"]["knowncluster"]["mibig_entries"]["1"]

                    # Dict for each ctg_tagid
                    blast_ = {}
                    for blast in blast_info:
                        ctg_name = blast[0].split("|")[4]
                        blast_["ctg_cordinates"] = blast[0].split("|")[2]
                        blast_name, blast_genecluster, blast_start, blast_end, blast_strand, blast_annotation, blast_perc_ident, blast_blastscore, blast_perc_coverage, blast_evalue, blast_locus_tag  = blast[2].values()
                        blast_[ctg_name] = {"blast_name" : blast_name, 
                                            "blast_genecluster": blast_genecluster,
                                            "blast_annotation": blast_annotation,
                                            "blast_perc_coverage":blast_perc_coverage,
                                            "blast_perc_ident": blast_perc_ident,
                                            "blast_blastscore": blast_blastscore,
                                            "blast_evalue": blast_evalue,
                                            "blast_locus_tag": blast_locus_tag,
                                            "cordinates": blast[0].split("|")[2] }

                    blast_keys = list(blast_.keys())[1:]
                    mibig_entries_keys = [ ctg_tagid for ctg_tagid in mibig_entries.keys()]
        

                    for ix, vl in  enumerate(blast_keys):
                        ctg_id = vl.split("_")[0]
                        ctg_tagid = vl
                        ctg_cordinates = blast_[vl]["cordinates"]
                        blast_name = blast_[vl]["blast_name"]
                        blast_genecluster= blast_[vl]["blast_genecluster"]
                        blast_annotation = blast_[vl]["blast_annotation"]
                        blast_perc_coverage = blast_[vl]["blast_perc_coverage"]
                        blast_perc_ident = blast_[vl]["blast_perc_ident"]
                        blast_blastscore = blast_[vl]["blast_blastscore"]
                        blast_evalue = blast_[vl]["blast_evalue"]
                        blast_locus_tag = blast_[vl]["blast_locus_tag"]
                        biosynthetic_gene_cluster_id=None 
                        mibig_entries_product=None

                        if vl not in mibig_entries_keys:
                            writer.writerow([
                                    bin_id, 
                                    contig_id,
                                    ctg_cordinates, 
                                    ctg_id, 
                                    ctg_tagid, 
                                    biosynthetic_gene_cluster_id, 
                                    mibig_entries_product, 
                                    blast_name,
                                    blast_genecluster,
                                    blast_annotation,
                                    blast_perc_coverage,
                                    blast_perc_ident,
                                    blast_blastscore,
                                    blast_evalue,
                                    blast_locus_tag
                                ])



                    for ctg_tagid, vl in mibig_entries.items():
                        ctg_id = ctg_tagid.split("_")[0]
                        mibig_entry = vl[0]
                        biosynthetic_gene_cluster_id = mibig_entry[2]
                        mibig_entries_product = mibig_entry[4]
                        blast_info_ = blast_.get(ctg_tagid)

                        if blast_info_:
                            ctg_cordinates = blast_info_["cordinates"]
                            blast_name = blast_info_["blast_name"]
                            blast_genecluster= blast_info_["blast_genecluster"]
                            blast_annotation = blast_info_["blast_annotation"]
                            blast_perc_coverage = blast_info_["blast_perc_coverage"]
                            blast_perc_ident = blast_info_["blast_perc_ident"]
                            blast_blastscore = blast_info_["blast_blastscore"]
                            blast_evalue = blast_info_["blast_evalue"]
                            blast_locus_tag = blast_info_["blast_locus_tag"]

                        else:
                            ctg_cordinates = None
                            blast_name = None
                            blast_genecluster= None
                            blast_annotation = None
                            blast_perc_coverage = None
                            blast_perc_ident = None
                            blast_blastscore = None
                            blast_evalue = None
                            blast_locus_tag = None
                            
                        writer.writerow([
                            bin_id, 
                            contig_id,
                            ctg_cordinates, 
                            ctg_id, 
                            ctg_tagid, 
                            biosynthetic_gene_cluster_id, 
                            mibig_entries_product, 
                            blast_name,
                            blast_genecluster,
                            blast_annotation,
                            blast_perc_coverage,
                            blast_perc_ident,
                            blast_blastscore,
                            blast_evalue,
                            blast_locus_tag
                        ])
            

process_files(list_dir)

In [34]:
import pandas as pd
dataframe = pd.read_csv("./comb_output.csv")
dataframe.head(100)

,Bin_ID,Contig_ID,Gene_Coordinates,Ctg_Number,Ctg_Tag_Id,Mibig_biosynthetic_gene_cluster,Mibig_Product,Blast_name,Blast_genecluster,Blast_annotation,Blast_perc_coverage,Blast_perc_ident,Blast_blastscore,Blast_evalue,Blast_locus_tag
0,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,29585-29864,ctg3,ctg3_29,NaN,NaN,CG392_RS04935,NZ_NNRS01000004_c78906-120244,hypothetical_protein,98.913043,100.0,186.0,2.970000e-61,WP_116438264.1
1,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,29958-30246,ctg3,ctg3_30,NaN,NaN,CG392_RS04940,NZ_NNRS01000004_c78906-120244,type_II_toxin-antitoxin_system_RelB/DinJ_famil...,100.000000,100.0,192.0,1.900000e-63,WP_116438265.1
2,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,30616-31054,ctg3,ctg3_31,NaN,NaN,CG392_RS04945,NZ_NNRS01000004_c78906-120244,ORF6N_domain-containing_protein,79.310345,74.0,160.0,5.670000e-50,CG392_RS04945
3,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,31058-32072,ctg3,ctg3_32,NaN,NaN,CG392_RS04950,NZ_NNRS01000004_c78906-120244,Abi_family_protein,100.000000,100.0,679.0,2.570000e-248,WP_116438267.1
4,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,33418-38701,ctg3,ctg3_34,NaN,NaN,CG392_RS07955,NZ_NNRS01000004_c78906-120244,hypothetical_protein,45.397727,45.0,571.0,6.240000e-181,CG392_RS07955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,MG_323757.bin1.fa,NODE_83_length_8923_cov_7.096640,813-996,ctg59,ctg59_2,NaN,NaN,KW861_RS14645,NZ_CP078505_c3139533-3154876,Blp_family_class_II_bacteriocin,100.000000,83.0,80.0,5.870000e-20,WP_103301327.1
90,MG_323757.bin1.fa,NODE_83_length_8923_cov_7.096640,NaN,ctg59,ctg59_6,BGC0000100,Polyketide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,MG_323757.bin1.fa,NODE_110_length_7016_cov_6.405832,520-1279,ctg82,ctg82_1,BGC0002667,RiPP,A5816_RS11485,NZ_NGLI01000002_c555810-572363,LytTR_family_DNA-binding_domain-containing_pro...,98.809524,93.0,460.0,1.940000e-164,WP_010733065.1
92,MG_323757.bin1.fa,NODE_110_length_7016_cov_6.405832,1280-2594,ctg82,ctg82_2,BGC0000624,RiPP,A5816_RS11480,NZ_NGLI01000002_c555810-572363,GHKL_domain-containing_protein,72.311213,88.0,527.0,6.040000e-187,WP_080114483.1


In [7]:
dataframe["Mibig_Product"].unique()

array(['RiPP:Microcin', 'RiPP:Thiopeptide', 'Other', 'Alkaloid', 'NRP',
       'NRP+Polyketide:Modular type I polyketide', 'RiPP:Lanthipeptide',
       'NRP+Polyketide', 'Polyketide', 'Saccharide',
       'Terpene+Polyketide:Type III polyketide', 'RiPP'], dtype=object)

In [14]:
# from sklearn.preprocessing import LabelEncoder
# dataframe_copy = dataframe.copy()

# # Initialize LabelEncoder
# label_encoder = LabelEncoder()

# # Encode the 'Category' column
# dataframe_copy['Mibig_Product_Encoded'] = label_encoder.fit_transform(dataframe_copy['Mibig_Product'])

# # Print the encoding mappings
# print("Encoding mappings:")
# for label, encoded_value in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)):
#     print(f"{label}: {encoded_value}")

# print("\nEncoded DataFrame:")
# print(dataframe_copy[['Mibig_Product', 'Mibig_Product_Encoded']])


# Going Modular

In [43]:
%%writefile python_scripts/json_csv.py 
import json
import csv
import os
import argparse

# Reads all of the json files in the dir
def list_json_files(directory):
    json_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.json'):
                json_files.append(os.path.join(root, file))
    return json_files

def above_two(data_records):
    index=[]
    for i in range(len(data_records)):
        modules = data_records[i]["modules"].keys()
        if len(modules) > 2:
            index.append(i)
        else:
            continue
    return index

def process_files(file_paths, csv_filename):
    
    # Check if the output file already exists
    file_exists = os.path.isfile(csv_filename)
    
    with open(csv_filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        
        if not file_exists:
            writer.writerow(["Bin_ID", "Contig_ID", "Gene_Coordinates", "Ctg_Number", "Ctg_Tag_Id", "Mibig_biosynthetic_gene_cluster", "Mibig_Product", "Blast_name", "Blast_genecluster","Blast_annotation","Blast_perc_coverage", "Blast_perc_ident", "Blast_blastscore", "Blast_evalue", "Blast_locus_tag" ])
        
        for file_path in file_paths:
            with open(file_path, 'r') as files:
                data = json.load(files)
                index_ = above_two(data["records"])

                for i in index_:
                    bin_id = data["input_file"]
                    contig_id = data["records"][i]["modules"]["antismash.modules.clusterblast"]["record_id"]
                    blast_info = data["records"][i]["modules"]["antismash.modules.clusterblast"]["general"]["results"][0]["ranking"][1][1]["pairings"]
                    mibig_entries = data["records"][i]["modules"]["antismash.modules.clusterblast"]["knowncluster"]["mibig_entries"]["1"]

                    # Dict for each ctg_tagid
                    blast_ = {}
                    for blast in blast_info:
                        ctg_name = blast[0].split("|")[4]
                        blast_["ctg_cordinates"] = blast[0].split("|")[2]
                        blast_name, blast_genecluster, blast_start, blast_end, blast_strand, blast_annotation, blast_perc_ident, blast_blastscore, blast_perc_coverage, blast_evalue, blast_locus_tag  = blast[2].values()
                        blast_[ctg_name] = {"blast_name" : blast_name, 
                                            "blast_genecluster": blast_genecluster,
                                            "blast_annotation": blast_annotation,
                                            "blast_perc_coverage":blast_perc_coverage,
                                            "blast_perc_ident": blast_perc_ident,
                                            "blast_blastscore": blast_blastscore,
                                            "blast_evalue": blast_evalue,
                                            "blast_locus_tag": blast_locus_tag,
                                            "cordinates": blast[0].split("|")[2] }

                    for ctg_tagid, vl in mibig_entries.items():
                        ctg_id = ctg_tagid.split("_")[0]
                        mibig_entry = vl[0]
                        biosynthetic_gene_cluster_id = mibig_entry[2]
                        mibig_entries_product = mibig_entry[4]
                        blast_info_ = blast_.get(ctg_tagid)

                        if blast_info_:
                            ctg_cordinates = blast_info_["cordinates"]
                            blast_name = blast_info_["blast_name"]
                            blast_genecluster= blast_info_["blast_genecluster"]
                            blast_annotation = blast_info_["blast_annotation"]
                            blast_perc_coverage = blast_info_["blast_perc_coverage"]
                            blast_perc_ident = blast_info_["blast_perc_ident"]
                            blast_blastscore = blast_info_["blast_blastscore"]
                            blast_evalue = blast_info_["blast_evalue"]
                            blast_locus_tag = blast_info_["blast_locus_tag"]

                        else:
                            ctg_cordinates = None
                            blast_name = None
                            blast_genecluster= None
                            blast_annotation = None
                            blast_perc_coverage = None
                            blast_perc_ident = None
                            blast_blastscore = None
                            blast_evalue = None
                            blast_locus_tag = None
                            
                        writer.writerow([
                            bin_id, 
                            contig_id,
                            ctg_cordinates, 
                            ctg_id, 
                            ctg_tagid, 
                            biosynthetic_gene_cluster_id, 
                            mibig_entries_product, 
                            blast_name,
                            blast_genecluster,
                            blast_annotation,
                            blast_perc_coverage,
                            blast_perc_ident,
                            blast_blastscore,
                            blast_evalue,
                            blast_locus_tag
                        ])
            
def main():
    parser = argparse.ArgumentParser(description="Process JSON files and output combined results to a CSV file. Please note dont run with the same output filename it will add the same results in the file.")
    parser.add_argument('--result_dir',"-r", help='JSON files to process')
    parser.add_argument('-o', '--output', default='combined_output.csv', help='Output CSV file name (default: combined_output.csv)')
    
    args = parser.parse_args()
    
    list_jsons = list_json_files(args.result_dir)

    process_files(list_jsons, args.output)
    print(f"Output written to: {args.output}")

if __name__ == "__main__":
    main()

Overwriting python_scripts/json_csv.py


In [21]:
!python3 python_scripts/json_csv.py -r "./results" -o "test_going_modular.csv"

Output written to: test_going_modular.csv


In [41]:
!python3 python_scripts/json_csv.py --help

usage: json_csv.py [-h] [--result_dir RESULT_DIR] [-o OUTPUT]

Process JSON files and output combined results to a CSV file. Please note dont
run with the same output name file it will add the same results in the file.

options:
  -h, --help            show this help message and exit
  --result_dir RESULT_DIR, -r RESULT_DIR
                        JSON files to process
  -o OUTPUT, --output OUTPUT
                        Output CSV file name (default: combined_output.csv)


In [1]:
import pandas as pd
results = pd.read_csv("./test_going_modular.csv")

results.head()

,Bin_ID,Contig_ID,Gene_Coordinates,Ctg_Number,Ctg_Tag_Id,Mibig_biosynthetic_gene_cluster,Mibig_Product,Blast_name,Blast_genecluster,Blast_annotation,Blast_perc_coverage,Blast_perc_ident,Blast_blastscore,Blast_evalue,Blast_locus_tag
0,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,NaN,ctg3,ctg3_27,BGC0000588,RiPP:Microcin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,28598-29438,ctg3,ctg3_28,BGC0000607,RiPP:Thiopeptide,CG392_RS04930,NZ_NNRS01000004_c78906-120244,AAA_family_ATPase,99.641577,100.0,536.0,1.450000e-193,WP_004121099.1
2,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,43941-44757,ctg3,ctg3_38,BGC0002284,Other,CG392_RS04985,NZ_NNRS01000004_c78906-120244,(p)ppGpp_synthetase,100.000000,95.0,493.0,8.280000e-177,WP_116438271.1
3,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,47230-48667,ctg3,ctg3_42,BGC0002561,Alkaloid,CG392_RS05005,NZ_NNRS01000004_c78906-120244,hydroxymethylglutaryl-CoA_synthase,101.673640,84.0,765.0,4.340000e-278,WP_116438274.1
4,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,50447-51650,ctg3,ctg3_44,BGC0000867,Other,CG392_RS05015,NZ_NNRS01000004_c78906-120244,acetyl-CoA_C-acyltransferase,106.500000,91.0,727.0,9.470000e-265,WP_116438276.1


In [4]:
results["Bin_ID"].unique()

array(['MG_323829.bin2.fa', 'MG_323757.bin3.fa', 'MG_323829.bin1.fa',
       'MG_323757.bin1.fa'], dtype=object)

# Going Modular Include Blast

In [39]:
%%writefile python_scripts/json_csv_with_blast.py 
import json
import csv
import os
import argparse

def above_two(data_records):
    index=[]
    for i in range(len(data_records)):
        modules = data_records[i]["modules"].keys()
        if len(modules) > 2:
            # print(i, len(modules))
            index.append(i)
        else:
            continue
    return index

# Reads all of the json files in the dir
def list_json_files(directory):
    json_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.json'):
                json_files.append(os.path.join(root, file))
    return json_files

def process_files(file_paths, csv_filename):
    
    # Check if the output file already exists
    file_exists = os.path.isfile(csv_filename)
    
    with open(csv_filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        
        if not file_exists:
            writer.writerow(["Bin_ID", "Contig_ID", "Gene_Coordinates", "Ctg_Number", "Ctg_Tag_Id", "Mibig_biosynthetic_gene_cluster", "Mibig_Product", "Blast_name", "Blast_genecluster","Blast_annotation","Blast_perc_coverage", "Blast_perc_ident", "Blast_blastscore", "Blast_evalue", "Blast_locus_tag" ])
        
        for file_path in file_paths:

            with open(file_path, 'r') as files:
                data = json.load(files)
                index_ = above_two(data["records"])
                
                for i in index_:
                    bin_id = data["input_file"]
                    contig_id = data["records"][i]["modules"]["antismash.modules.clusterblast"]["record_id"]
                    blast_info = data["records"][i]["modules"]["antismash.modules.clusterblast"]["general"]["results"][0]["ranking"][1][1]["pairings"]
                    mibig_entries = data["records"][i]["modules"]["antismash.modules.clusterblast"]["knowncluster"]["mibig_entries"]["1"]

                    # Dict for each ctg_tagid
                    blast_ = {}
                    for blast in blast_info:
                        ctg_name = blast[0].split("|")[4]
                        blast_["ctg_cordinates"] = blast[0].split("|")[2]
                        blast_name, blast_genecluster, blast_start, blast_end, blast_strand, blast_annotation, blast_perc_ident, blast_blastscore, blast_perc_coverage, blast_evalue, blast_locus_tag  = blast[2].values()
                        blast_[ctg_name] = {"blast_name" : blast_name, 
                                            "blast_genecluster": blast_genecluster,
                                            "blast_annotation": blast_annotation,
                                            "blast_perc_coverage":blast_perc_coverage,
                                            "blast_perc_ident": blast_perc_ident,
                                            "blast_blastscore": blast_blastscore,
                                            "blast_evalue": blast_evalue,
                                            "blast_locus_tag": blast_locus_tag,
                                            "cordinates": blast[0].split("|")[2] }

                    blast_keys = list(blast_.keys())[1:]
                    mibig_entries_keys = [ ctg_tagid for ctg_tagid in mibig_entries.keys()]
        

                    for ix, vl in  enumerate(blast_keys):
                        ctg_id = vl.split("_")[0]
                        ctg_tagid = vl
                        ctg_cordinates = blast_[vl]["cordinates"]
                        blast_name = blast_[vl]["blast_name"]
                        blast_genecluster= blast_[vl]["blast_genecluster"]
                        blast_annotation = blast_[vl]["blast_annotation"]
                        blast_perc_coverage = blast_[vl]["blast_perc_coverage"]
                        blast_perc_ident = blast_[vl]["blast_perc_ident"]
                        blast_blastscore = blast_[vl]["blast_blastscore"]
                        blast_evalue = blast_[vl]["blast_evalue"]
                        blast_locus_tag = blast_[vl]["blast_locus_tag"]
                        biosynthetic_gene_cluster_id=None 
                        mibig_entries_product=None

                        if vl not in mibig_entries_keys:
                            writer.writerow([
                                    bin_id, 
                                    contig_id,
                                    ctg_cordinates, 
                                    ctg_id, 
                                    ctg_tagid, 
                                    biosynthetic_gene_cluster_id, 
                                    mibig_entries_product, 
                                    blast_name,
                                    blast_genecluster,
                                    blast_annotation,
                                    blast_perc_coverage,
                                    blast_perc_ident,
                                    blast_blastscore,
                                    blast_evalue,
                                    blast_locus_tag
                                ])



                    for ctg_tagid, vl in mibig_entries.items():
                        ctg_id = ctg_tagid.split("_")[0]
                        mibig_entry = vl[0]
                        biosynthetic_gene_cluster_id = mibig_entry[2]
                        mibig_entries_product = mibig_entry[4]
                        blast_info_ = blast_.get(ctg_tagid)

                        if blast_info_:
                            ctg_cordinates = blast_info_["cordinates"]
                            blast_name = blast_info_["blast_name"]
                            blast_genecluster= blast_info_["blast_genecluster"]
                            blast_annotation = blast_info_["blast_annotation"]
                            blast_perc_coverage = blast_info_["blast_perc_coverage"]
                            blast_perc_ident = blast_info_["blast_perc_ident"]
                            blast_blastscore = blast_info_["blast_blastscore"]
                            blast_evalue = blast_info_["blast_evalue"]
                            blast_locus_tag = blast_info_["blast_locus_tag"]

                        else:
                            ctg_cordinates = None
                            blast_name = None
                            blast_genecluster= None
                            blast_annotation = None
                            blast_perc_coverage = None
                            blast_perc_ident = None
                            blast_blastscore = None
                            blast_evalue = None
                            blast_locus_tag = None
                            
                        writer.writerow([
                            bin_id, 
                            contig_id,
                            ctg_cordinates, 
                            ctg_id, 
                            ctg_tagid, 
                            biosynthetic_gene_cluster_id, 
                            mibig_entries_product, 
                            blast_name,
                            blast_genecluster,
                            blast_annotation,
                            blast_perc_coverage,
                            blast_perc_ident,
                            blast_blastscore,
                            blast_evalue,
                            blast_locus_tag
                        ])

def main():
    parser = argparse.ArgumentParser(description="Process JSON files and output combined results to a CSV file. Please note dont run with the same output filename it will add the same results in the file.")
    parser.add_argument('--result_dir',"-r", help='JSON files to process')
    parser.add_argument('-o', '--output', default='combined_output.csv', help='Output CSV file name (default: combined_output.csv)')
    
    args = parser.parse_args()
    
    list_jsons = list_json_files(args.result_dir)

    process_files(list_jsons, args.output)
    print(f"Output written to: {args.output}")

if __name__ == "__main__":
    main()

Overwriting python_scripts/json_csv_with_blast.py


In [40]:
!python3 python_scripts/json_csv_with_blast.py -r "./results" -o "output_with_blast.csv"

Output written to: output_with_blast.csv


In [4]:
import pandas as pd
df= pd.read_csv("./output_with_blast.csv")

df.head(1000)

,Bin_ID,Contig_ID,Gene_Coordinates,Ctg_Number,Ctg_Tag_Id,Mibig_biosynthetic_gene_cluster,Mibig_Product,Blast_name,Blast_genecluster,Blast_annotation,Blast_perc_coverage,Blast_perc_ident,Blast_blastscore,Blast_evalue,Blast_locus_tag
0,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,29585-29864,ctg3,ctg3_29,NaN,NaN,CG392_RS04935,NZ_NNRS01000004_c78906-120244,hypothetical_protein,98.913043,100.0,186.0,2.970000e-61,WP_116438264.1
1,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,29958-30246,ctg3,ctg3_30,NaN,NaN,CG392_RS04940,NZ_NNRS01000004_c78906-120244,type_II_toxin-antitoxin_system_RelB/DinJ_famil...,100.000000,100.0,192.0,1.900000e-63,WP_116438265.1
2,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,30616-31054,ctg3,ctg3_31,NaN,NaN,CG392_RS04945,NZ_NNRS01000004_c78906-120244,ORF6N_domain-containing_protein,79.310345,74.0,160.0,5.670000e-50,CG392_RS04945
3,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,31058-32072,ctg3,ctg3_32,NaN,NaN,CG392_RS04950,NZ_NNRS01000004_c78906-120244,Abi_family_protein,100.000000,100.0,679.0,2.570000e-248,WP_116438267.1
4,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,33418-38701,ctg3,ctg3_34,NaN,NaN,CG392_RS07955,NZ_NNRS01000004_c78906-120244,hypothetical_protein,45.397727,45.0,571.0,6.240000e-181,CG392_RS07955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,MG_323757.bin1.fa,NODE_83_length_8923_cov_7.096640,813-996,ctg59,ctg59_2,NaN,NaN,KW861_RS14645,NZ_CP078505_c3139533-3154876,Blp_family_class_II_bacteriocin,100.000000,83.0,80.0,5.870000e-20,WP_103301327.1
90,MG_323757.bin1.fa,NODE_83_length_8923_cov_7.096640,NaN,ctg59,ctg59_6,BGC0000100,Polyketide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,MG_323757.bin1.fa,NODE_110_length_7016_cov_6.405832,520-1279,ctg82,ctg82_1,BGC0002667,RiPP,A5816_RS11485,NZ_NGLI01000002_c555810-572363,LytTR_family_DNA-binding_domain-containing_pro...,98.809524,93.0,460.0,1.940000e-164,WP_010733065.1
92,MG_323757.bin1.fa,NODE_110_length_7016_cov_6.405832,1280-2594,ctg82,ctg82_2,BGC0000624,RiPP,A5816_RS11480,NZ_NGLI01000002_c555810-572363,GHKL_domain-containing_protein,72.311213,88.0,527.0,6.040000e-187,WP_080114483.1


In [44]:
df.shape

(94, 15)

# Add Taxanom

In [2]:
import pandas as pd
import csv
import sys

def convert_txt_to_csv(input_file, output_file):
    try:
        with open(input_file, 'r', newline='') as infile, open(output_file, 'w', newline='') as outfile:
            reader = csv.reader(infile, delimiter='\t')
            writer = csv.writer(outfile)

            # Write all rows to the CSV file
            for row in reader:
                writer.writerow(row)

        print(f"Successfully converted {input_file} to {output_file}")
    except Exception as e:
        print(f"An error occurred: {e}")
        sys.exit(1)

if __name__ == "__main__":
    input_file = "MAG_inventory_VIRGO2.txt" 
    output_file = "MAG_inventory_VIRGO2.csv"  

    convert_txt_to_csv(input_file, output_file)

Successfully converted MAG_inventory_VIRGO2.txt to MAG_inventory_VIRGO2.csv


In [8]:
mag_virgo = pd.read_csv("MAG_inventory_VIRGO2.csv")
mag_virgo.head()

,MAG,MAG_originalName,Metagenome,Project,Taxonomy,FilterContam,KeepCleaned,R1Complete,R2Complete,ComplDif,...,closest_placement_ani,closest_placement_af,pplacer_taxonomy,classification_method,note,"other_related_references(genome_id,species_name,radius,ANI,AF)",msa_percent,translation_table,red_value,warnings
0,MAG00001,MG_0052RFLQV1.bin1,MG_0052RFLQV1,XPENN,Lactobacillus_crispatus,NaN,0,98.86,98.86,0.00,...,97.56,0.70,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,taxonomic classification defined by topology a...,topological placement and ANI have congruent s...,"GCF_900103655.1, s__Lactobacillus kefiranofaci...",90.27,11,NaN,NaN
1,MAG00002,MG_0139RFLQV1.bin1,MG_0139RFLQV1,XPENN,Lactobacillus_mulieris,NaN,0,87.28,85.74,-1.54,...,96.99,0.91,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,taxonomic classification defined by topology a...,topological placement and ANI have congruent s...,"GCF_001436455.1, s__Lactobacillus jensenii, 95...",87.43,11,NaN,NaN
2,MAG00003,MG_0139RFLQV1.bin2,MG_0139RFLQV1,XPENN,Bradyrhizobium_sp003020075,1.0,0,80.75,79.86,-0.89,...,99.77,0.96,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,taxonomic classification defined by topology a...,topological placement and ANI have congruent s...,"GCA_001238275.1, s__Bradyrhizobium viridifutur...",62.71,11,NaN,NaN
3,MAG00004,MG_0139RFLQV1.bin3,MG_0139RFLQV1,XPENN,Lactobacillus_iners,NaN,0,94.01,94.02,0.01,...,98.73,0.94,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,taxonomic classification defined by topology a...,topological placement and ANI have congruent s...,"GCF_003423665.1, s__Lactobacillus rodentium, 9...",91.32,11,NaN,NaN
4,MAG00005,MG_0226RFLQV1.bin1,MG_0226RFLQV1,XPENN,Cutibacterium_modestum,NaN,1,83.47,82.60,-0.87,...,99.91,1.00,d__Bacteria;p__Actinobacteriota;c__Actinomycet...,taxonomic classification defined by topology a...,topological placement and ANI have congruent s...,"GCF_003030305.1, s__Cutibacterium acnes, 95.0,...",62.59,11,NaN,NaN


In [31]:
# Create new column MAG_originalName on the json results dataframe
df["MAG_originalName"] = df["Bin_ID"].map(lambda x: x.split(".")[0]+ "." + x.split(".")[1])

# Merge the MAG_inventory_VIRGO2 on MAG_originalName with the json results
df_results_tax =  pd.merge(df, mag_virgo[['MAG_originalName', 'Taxonomy']], on='MAG_originalName', how='inner')

In [35]:
df_results_tax.Taxonomy.unique()

array(['Gardnerella_piotii', 'Lactobacillus_iners',
       'Streptococcus_agalactiae', 'Lactobacillus_mulieris'], dtype=object)

In [41]:
df_results_tax[["Taxonomy", "Mibig_Product"]]

,Taxonomy,Mibig_Product
0,Gardnerella_piotii,NaN
1,Gardnerella_piotii,NaN
2,Gardnerella_piotii,NaN
3,Gardnerella_piotii,NaN
4,Gardnerella_piotii,NaN
...,...,...
89,Lactobacillus_mulieris,NaN
90,Lactobacillus_mulieris,Polyketide
91,Lactobacillus_mulieris,RiPP
92,Lactobacillus_mulieris,RiPP
